In [8]:
from pynq import Overlay, allocate
import numpy as np
import time

# Load the overlay (replace with the correct path if needed)
ol = Overlay("sistema.bit")

In [9]:
dir(ol)

['PSDDR',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_deepcopy_dict_from',
 '_ip_map',
 '_register_drivers',
 'array_gain_0',
 'axi_intc_0',
 'binfile_name',
 'bit_data',
 'bitfile_name',
 'clock_dict',
 'device',
 'download',
 'dtbo',
 'firmware_path',
 'free',
 'gen_cache',
 'gpio_dict',
 'hierarchy_dict',
 'ignore_version',
 'insert_dtbo',
 'interrupt_controllers',
 'interrupt_pins',
 'ip_dict',
 'is_loaded',
 'load_ip_data',
 'mem_dict',
 'parser',
 'partial',
 'pr_dict',
 'pr_download',
 'remove_dtbo',
 'reset',
 'systemgraph',
 'timestamp',
 'zynq_ultra_ps_e_0']

In [10]:
help(ol.free)

Help on method free in module pynq.overlay:

free() method of pynq.overlay.Overlay instance



In [11]:
# Define the array size: 2^13 = 8192 elements
size = 8192

# Allocate contiguous memory for input and output buffers
input_buffer = allocate(shape=(size,), dtype=np.float32)
output_buffer = allocate(shape=(size,), dtype=np.float32)

# Initialize input data (for example, with random numbers)
input_buffer[:] = np.random.rand(size).astype(np.float32)
gain = np.float32(2.0)  # Set your gain value

In [12]:
kr = ol.array_gain_0
kr.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  input_r_1 = Register(input_r=write-only),
  input_r_2 = Register(input_r=write-only),
  output_r_1 = Register(output_r=write-only),
  output_r_2 = Register(output_r=write-only),
  gain = Register(gain=write-only)
}

In [13]:
# Write the physical addresses and parameter to the registers:
kr.write(0x10, input_buffer.physical_address)

# Convert the float 'gain' into its 32-bit integer representation and cast to int:
gain_int = int(np.frombuffer(gain.tobytes(), dtype=np.uint32)[0])
kr.write(0x28, gain_int)

kr.write(0x1c, output_buffer.physical_address)

# Start the kernel by writing '1' to the control register at offset 0x00
kr.write(0x00, 0x01)

# Poll the control register until the kernel sets the "done" flag (usually bit 1)
while (kr.read(0x00) & 0x2) == 0:
    time.sleep(0.001)

# Now, output_buffer should contain the results (each element multiplied by 'gain')
print("First 10 input values  :", input_buffer[:10])
print("First 10 output values :", output_buffer[:10])
print("Gain used              :", gain)

# When finished, free the allocated buffers (if desired)
input_buffer.freebuffer()
output_buffer.freebuffer()

First 10 input values  : [0.331142   0.8920735  0.8433798  0.82268065 0.28207535 0.7442016
 0.30375603 0.69301593 0.28749317 0.6999363 ]
First 10 output values : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Gain used              : 2.0


In [14]:
hex(kr.register_map.gain.address)

'0x28'

In [15]:
hex(kr.register_map.input_r_1.address)

'0x10'

In [16]:
hex(kr.register_map.output_r_1.address)

'0x1c'